# Analyzing and Predicting Player Count Trends in Online Games

## Part 1: An Introduction

As we enter the second decade of the 21st century, the interconnectedness of our world increases every day. Once, activities done alone are now conducted online with fellows, and video games are no different. An estimated 2.7 billion gamers exist in the world, and this number can only increase due to the impact COVID-19 has had, and will continue to have, on our culture. With that in mind, businesses that serve these customers must be able to handle the changes that surround us. They must effectively predict how their customer base will be changing in the long run to address capital restrictions, how their player base changes over the medium run to address capturing the largest player base for events/announcements, and hwo their player base changes over the short run to to address how employing short-term retention strategies. 

## Our Project
We will be using data on concurrent players for multiple games in order to predict number of active players. We can accomplish this on a long term, medium term, and short term basis to help us answer business questions that can ecompass all time periods. Breaking these investigations down, we will:
- Make a long-run (LR) prediction for player base, which will help us decide how a business should increase or decrease their capital, namely servers, the primary capital for most online game businesses. 
- Make a medium-run (MR) prediction for player base, which will help us show when most players are online over the course of a year. Knowing yearly trends will assist in planning events for players, which will lead to increased player retention. 
- Make a short-run (SR) prediction model for player base, which will help show short-term trends. This will show businesses where short term retention strategies would be most applicable. 

### Project Map
First, we will need to collect data to make our predictions, and this will come from Steam, the market dominator for PC gaming. While this will not capture the full sample space of gamers worldwide, Steam is an excellent choice for our data source, as most online gamers opt for playing on PC rather than console interfaces, and Steam dominates the PC gaming market by a great margin. 

Our data will be structured quite simply. Raw data contains the date, number of players during that day, any flags that are associated with the date (events usually), and number of twitch viewers at that time. We will be dropping number of twitch viewers for our modeling phase, as this is not helpful in predicting number of players. However, taking twitch viewers into account during our EDA phase may be useful in giving richer context. 

We will have to build models for multiple games in order to avoid overfitting our model to a single game. While a single business may wish to have a model that is precisely fitted to their product, we are attempting to construct a model that can be applied to any online game. 

During our modelling phase, we will need to adjust parameters to account for the different time periods we wish to predict. In order to do this, we will be using SARIMA or SARIMA EXR, two types of supervised learning models that account for seasonal trends in data. When we model, our "seasons" will adjust to the time period we are analyzing: LR is yearly, MR is monthly, and SR is weekly. 

## Data Collection
We established criteria for selecting games from steamdb.info. Our selected games to investigate should:
1. Be online. 
2. Be popular.
3. Not be a new release.
4. Have in-game events, seasonal or non-seasonal
5. Have in-game rewards
6. Be available worldwide. 

While it is not imperative for our games to meet all of these criteria, the more that are met, the better we can apply the given game to our investigation. After some initial research on SteamDB, the following games meet most of our criteria, and will be used for our investigation:

- Rocket League, 2015
- Counter Strike: Global Offensive, 2012
- DOTA 2, 2013
- Team Fortress 2, 2007 *

*Team Fortress 2 has limited events and rewards, but has an extremely large sample space, so it acts as a great control compared to the other games selected.

## Raw Data

This section serves to introduce our data, make some basic alterations for ease of use, and complete introductory feature engineering. 

In [54]:
# we begin with importing introductory libraries

!pip install -U fsds

from fsds.imports import * 

Requirement already up-to-date: fsds in c:\users\rmcar\anaconda\envs\learn-env\lib\site-packages (0.2.28)


In [55]:
# we import our raw data

dota = pd.read_csv('data/Raw/dota2.csv')
csgo = pd.read_csv('data/Raw/csgo.csv')
rl = pd.read_csv('data/Raw/rl.csv')
tf = pd.read_csv('data/Raw/tf2.csv')

# we construct a list of these dataframes to use throughout

ls = [csgo, dota, rl, tf]

dota

,DateTime,Players,Flags,Twitch Viewers
0,2011-09-22 00:00:00,194.0,NaN,NaN
1,2011-09-23 00:00:00,240.0,NaN,NaN
2,2011-09-24 00:00:00,NaN,NaN,NaN
3,2011-09-25 00:00:00,233.0,NaN,NaN
4,2011-09-26 00:00:00,222.0,NaN,NaN
...,...,...,...,...
3387,2020-12-30 00:00:00,625344.0,NaN,71052.0
3388,2020-12-31 00:00:00,561914.0,NaN,60719.0
3389,2021-01-01 00:00:00,642032.0,NaN,57219.0
3390,2021-01-02 00:00:00,694687.0,NaN,63547.0


In [56]:
rl

,DateTime,Players,Flags,Twitch Viewers
0,2014-01-07 00:00:00,12.0,NaN,NaN
1,2014-01-08 00:00:00,NaN,NaN,NaN
2,2014-01-09 00:00:00,NaN,NaN,NaN
3,2014-01-10 00:00:00,NaN,NaN,NaN
4,2014-01-11 00:00:00,NaN,NaN,NaN
...,...,...,...,...
2549,2020-12-30 00:00:00,113708.0,NaN,89994.0
2550,2020-12-31 00:00:00,88266.0,NaN,53390.0
2551,2021-01-01 00:00:00,102974.0,NaN,72013.0
2552,2021-01-02 00:00:00,109847.0,NaN,83405.0


In [57]:
# we inspect columns
dota.columns

Index(['DateTime', 'Players', 'Flags', 'Twitch Viewers'], dtype='object')

### Flags
Upon inspecting this column, we were disappointed to see that event notes did not translate when importing our data, resulting in this column being empty. We will instead have to complete this manually from SteamDB in our feature engineering section. 

In [58]:
# we drop 'Flags'

ls = list(map(lambda df:df.drop(columns = 'Flags'), ls))
ls

[                 DateTime    Players  Twitch Viewers
 0     2011-11-30 00:00:00      680.0             NaN
 1     2011-12-01 00:00:00        NaN             NaN
 2     2011-12-02 00:00:00        NaN             NaN
 3     2011-12-03 00:00:00        NaN             NaN
 4     2011-12-04 00:00:00        NaN             NaN
 ...                   ...        ...             ...
 3318  2020-12-30 00:00:00  1056057.0         77815.0
 3319  2020-12-31 00:00:00   982583.0         60120.0
 3320  2021-01-01 00:00:00  1020715.0         91532.0
 3321  2021-01-02 00:00:00  1079804.0         93678.0
 3322  2021-01-03 00:00:00  1067795.0         94130.0
 
 [3323 rows x 3 columns],
                  DateTime   Players  Twitch Viewers
 0     2011-09-22 00:00:00     194.0             NaN
 1     2011-09-23 00:00:00     240.0             NaN
 2     2011-09-24 00:00:00       NaN             NaN
 3     2011-09-25 00:00:00     233.0             NaN
 4     2011-09-26 00:00:00     222.0             NaN
 ...  

A breakdown of the columns in our data:
- DateTime: Shows the date in format YYYY-MM-DD HH:MM:SS. However, data is collected on a daily basis at 00:00:00, so we will need to format this column so that only the date shows, and it is pandas encoded as well.
- Players: Shows the number of players during the day. This will be our target during our modelling phase. 
- Flags: A note column for the day that is recorded. Usually indicates an event.
- Twitch Viewers: Shows the number of twitch viewers during that day. 

In [59]:
# we rename columns

rename_dict = {
    "DateTime" : "time",
    "Players" : "players",
    "Twitch Viewers" : 'viewers'
}

ls = list(map(lambda df: df.rename(columns = rename_dict), ls))
ls

[                     time    players  viewers
 0     2011-11-30 00:00:00      680.0      NaN
 1     2011-12-01 00:00:00        NaN      NaN
 2     2011-12-02 00:00:00        NaN      NaN
 3     2011-12-03 00:00:00        NaN      NaN
 4     2011-12-04 00:00:00        NaN      NaN
 ...                   ...        ...      ...
 3318  2020-12-30 00:00:00  1056057.0  77815.0
 3319  2020-12-31 00:00:00   982583.0  60120.0
 3320  2021-01-01 00:00:00  1020715.0  91532.0
 3321  2021-01-02 00:00:00  1079804.0  93678.0
 3322  2021-01-03 00:00:00  1067795.0  94130.0
 
 [3323 rows x 3 columns],
                      time   players  viewers
 0     2011-09-22 00:00:00     194.0      NaN
 1     2011-09-23 00:00:00     240.0      NaN
 2     2011-09-24 00:00:00       NaN      NaN
 3     2011-09-25 00:00:00     233.0      NaN
 4     2011-09-26 00:00:00     222.0      NaN
 ...                   ...       ...      ...
 3387  2020-12-30 00:00:00  625344.0  71052.0
 3388  2020-12-31 00:00:00  561914.0  60

## Null Values

In [60]:
# we fill NaN with 0

ls = list(map(lambda df: df.fillna(0), ls))
ls[0]
    

,time,players,viewers
0,2011-11-30 00:00:00,680.0,0.0
1,2011-12-01 00:00:00,0.0,0.0
2,2011-12-02 00:00:00,0.0,0.0
3,2011-12-03 00:00:00,0.0,0.0
4,2011-12-04 00:00:00,0.0,0.0
...,...,...,...
3318,2020-12-30 00:00:00,1056057.0,77815.0
3319,2020-12-31 00:00:00,982583.0,60120.0
3320,2021-01-01 00:00:00,1020715.0,91532.0
3321,2021-01-02 00:00:00,1079804.0,93678.0


Our data contains player counts and viewers for times before a given game's release, so we will need to alter the bounds of our times in order to gain data that will be modellable. 

In [61]:
# we update time column to reflect datetime object type

for df in ls:
    for i, t in enumerate(df['time']):
        
        # format to get YYYYMMDDSSSSSS
        
        t = t.replace('-', '')
        t = t.replace(':', '')
        t = t.replace(' ', '')
        
        # remove seconds to get YYYYMMDD
        
        df['time'][i] = t[:-6]
        
    # convert to datetime
    
    df['time'] = pd.to_datetime(df['time'], format = '%Y%m%d')

In [62]:
# finally, we convert players and viewers from float to int

for df in ls:
    df['players'] = df['players'].astype(int)
    df['viewers'] = df['viewers'].astype(int)

In [63]:
csgo = ls[0].where(ls[0]['time'] >= "2015-01-01").dropna()
dota = ls[1].where(ls[1]['time'] >= "2015-01-01").dropna()
rl = ls[2].where(ls[2]['time'] >= "2015-07-27").dropna()
tf = ls[3].where(ls[3]['time'] >= "2012-01-01").dropna()

ls = [csgo, dota, rl, tf]
ls = list(map(lambda df: df.reset_index(), ls))

### Introductory Feature Engineering
There are some features that we need to address as soon as possible, though others will likely reveal themselves as we continue on. As stated earlier, we must manually address the issue of events not being encoded in our data. We will be constructing a column labelled 'event', encoded as '1' for an event ongoing, or '0' for no event. 

Furthermore, we should construct a column that represents the percent change in players and viewers, giving us a column that measures margins. 

#### Events
We made a csv file named 'events' that contains the date, and columns for each of our games events; 1 for an event ongoing, 0 for no event. You will notice that Team Fortress 2 is not included in this section, and that is because there were no events for the lifetime of Team Fortress 2.

In [64]:
# we construct our 'event' column. 

In [65]:
# we import the csv file we constructed for this process

df_events = pd.read_csv('data/Raw/events.csv').fillna(0)

df_events

,time,csgo,dota,rl
0,2011-11-30,0.0,0.0,0.0
1,2011-12-01,0.0,0.0,0.0
2,2011-12-02,0.0,0.0,0.0
3,2011-12-03,0.0,0.0,0.0
4,2011-12-04,0.0,0.0,0.0
...,...,...,...,...
3318,2020-12-30,0.0,0.0,0.0
3319,2020-12-31,0.0,0.0,0.0
3320,2021-01-01,0.0,0.0,0.0
3321,2021-01-02,0.0,0.0,0.0


In [66]:
# we format the 'time' column to datetime

for i in range(len(df_events['time'])):
    
    df_events['time'][i] = df_events['time'][i].replace('-', '') 
    
df_events['time'] = pd.to_datetime(df_events['time'], format = '%Y%m%d')

In [67]:
# we establish event df for each game

df_csgo = df_events.drop(columns = ['dota', 'rl'])
df_dota = df_events.drop(columns = ['csgo', 'rl'])
df_rl = df_events.drop(columns = ['dota', 'csgo'])


In [68]:
# and join the player count dataframes with these event dataframes

csgo_join = ls[0].merge(df_csgo, how = 'inner', on = 'time')
dota_join = ls[1].merge(df_dota, how = 'inner', on = 'time')
rl_join = ls[2].merge(df_rl, how = 'inner', on = 'time')


In [69]:
# finally, we rename the column to reflect 'event', we also need to add an event column for tf
# even though this column will be fully 0

csgo = csgo_join.rename(columns = {'csgo':'event'})
dota = dota_join.rename(columns = {'dota':'event'})
rl = rl_join.rename(columns = {'rl':'event'})

tf = ls[3]
tf['event'] = 0 


In [70]:
# we re-establish our list of dataframes

ls = [csgo, dota, rl, tf]

### Marginal change in Players/Viewers

This will be a helpful column to construct now, as inspecting the change in players/viewers is arguably more important than just inspecting the raw number of players/viewers. It will also normalize our data so that we can easily compare different games without worrying that the player bases are of such different scale. 

In [71]:
# we build a function that takes a dataframe and a column, and returns a list of the same length
# showing marginal changes.

def marginal_change(df, column):
    
    change_ls = []
    
    # loop through column
    
    for i in range(len(df)):
        
        # we need to skip i = 0 since there is no marginal change at the first row
        
        if i > 0:
            
            # we cannot divide by zero, so we must manually account for these instances
            
            if df[column][i-1] == 0:
                
                if df[column][i] == 0:
                    change_ls.append(0)
                    
                else:
                    change_ls.append(1)
            
            # straightforward change calculation
            
            else:
                change = df[column][i]/df[column][i-1] 
                change -= 1
                change_ls.append(change)
        else:
            change_ls.append(0)
        
    return change_ls
                  

In [72]:
# we construct the marginal columns for all dataframes

for df in ls:
    df['%chg_players'], df['%chg_viewers'] = (marginal_change(df, 'players'), 
                                              marginal_change(df, 'viewers'))
csgo    

,index,time,players,viewers,event,%chg_players,%chg_viewers
0,1128,2015-01-01,443188.0,0.0,0.0,0.000000,0.000000
1,1129,2015-01-02,383943.0,0.0,0.0,-0.133679,0.000000
2,1130,2015-01-03,398639.0,0.0,0.0,0.038277,0.000000
3,1131,2015-01-04,389040.0,0.0,0.0,-0.024079,0.000000
4,1132,2015-01-05,376364.0,0.0,0.0,-0.032583,0.000000
...,...,...,...,...,...,...,...
2190,3318,2020-12-30,1056057.0,77815.0,0.0,0.021949,-0.075634
2191,3319,2020-12-31,982583.0,60120.0,0.0,-0.069574,-0.227398
2192,3320,2021-01-01,1020715.0,91532.0,0.0,0.038808,0.522488
2193,3321,2021-01-02,1079804.0,93678.0,0.0,0.057890,0.023445


In this introductory section, we have:

* Cleaned data, including datatype handling, null value handling.
* We manually addressed the 'flag' column, which did not translate correctly from our data source. We constructed a csv file manually and imported this, joining with our dataframes. 
* We completed some basic feature engineering, giving us the percent change in both players and viewers, which will be extremely helpful in our EDA.

Next, we will be completing Exploratory Data Analysis, in which we will be exploring trends in our data, relationships between predictors, and much more. 

In [73]:
# we export our dataframes to use in the next notebook

csgo.to_csv('data/Clean/csgo.csv')
dota.to_csv('data/Clean/dota.csv')
rl.to_csv('data/Clean/rl.csv')
tf.to_csv('data/Clean/tf.csv')
